<a href="https://colab.research.google.com/github/Samridhi0705/MajorProject/blob/main/mp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()
import asyncio
asyncio.get_event_loop().run_until_complete(main())
!pip install telethon nest_asyncio


from telethon import TelegramClient, events

import os
if os.path.exists('bot_session.session'):
    os.remove('bot_session.session')

api_id = '28932578'
api_hash = 'e73796fdffd4d3fa973e1aedbe803311'
bot_token = '8315747892:AAHpfpYvvuC9lXBpycHHOBarC8_DTvOV0fs'

client = TelegramClient('bot_session', api_id, api_hash)

async def main():
    await client.start(bot_token=bot_token)

    @client.on(events.NewMessage)
    async def handler(event):
        print(f"New message: {event.message.message}")

    print("Bot is running...")
    await client.run_until_disconnected()

import asyncio
await main()  # This works in Colab/Jupyter
